In [11]:
from transformers import AutoTokenizer, AutoModel
import torch

# We should define the proper pooling function: Mean pooling
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ["This is an example sentence fuckity fuck"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('AIDA-UPM/mstsb-paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('AIDA-UPM/mstsb-paraphrase-multilingual-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    
res1 = model_output['pooler_output']
res2 = mean_pooling(model_output, encoded_input['attention_mask'])

print(res1[0, :10], res2[0, :10])

tensor([-0.0546,  0.0476, -0.0773,  0.0028, -0.0941,  0.0162,  0.0235,  0.0202,
        -0.0692,  0.0278]) tensor([ 0.0187, -0.2056, -0.0064, -0.0030,  0.0872,  0.1107,  0.0519, -0.0070,
         0.1441, -0.0186])
